## общее

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import os, sys

parent_dir = os.path.split(os.getcwd())[0]
if parent_dir not in sys.path:
    sys.path.append(parent_dir)

In [3]:
# целевая архитектура библиотеки - ЛД 3.0, поэтому используем Spark
# для отладки моделей (не на кластере) создаём локальную сессию
from sponge_bob_magic.session_handler import get_spark_session

spark = get_spark_session()
spark

## Подготовка данных <a name='data-preparator'></a>

In [4]:
# библиотека содержит утилиты для скачивания и парсинга популярных датасетов для рекомендаций
from sponge_bob_magic.datasets.movielens import MovieLens

data = MovieLens("1m")
data.ratings.to_csv("data.csv", index=False)
data.info()

ratings


,user_id,item_id,relevance,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968



users


,user_id,gender,age,occupation,zip_code
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117



items


,item_id,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance


In [5]:
# на данный момент модуль загрузки открытых данных не использует Spark
# в этой ячейке пример того, как можно использовать свои данные в формате CSV
from sponge_bob_magic.data_preparator.data_preparator import DataPreparator

log = DataPreparator(spark).transform_log(
    "data.csv",
    columns_names={
        "user_id": "user_id",
        "item_id": "item_id",
        "relevance": "relevance",
        "timestamp": "timestamp"
    },
    header=True,
    format_type="csv"
)

In [6]:
# библиотека содержит различные схемы валидации рекомендательных систем, встречающиеся в литературе
# пользователь может выбрать ту, которая лучше подходит бизнес-задаче
from sponge_bob_magic.splitters.user_log_splitter import ByTimeUserLogSplitter

splitter = ByTimeUserLogSplitter(
    drop_cold_items=True,
    drop_cold_users=True,
    item_test_size=1,
    user_test_size=500,
    seed=1234
)
train, test_input, test = splitter._split_quantity(log)
(
    train.count(), 
    test_input.count(), 
    test.count()
)

(999703, 999703, 506)

## NMF

In [7]:
# простейший пример использования DL в рекомендациях
from sponge_bob_magic.models.neuromf_recommender import NeuroMFRecommender

nmf = NeuroMFRecommender(
    learning_rate=0.01,
    epochs=10,
    embedding_dimension=100
)

In [8]:
%%time

nmf.fit(
    log=train,
    user_features=None,
    item_features=None
)

CPU times: user 14.6 s, sys: 2.8 s, total: 17.3 s
Wall time: 51.3 s


In [9]:
%%time

recs = nmf.predict(
    k=10,
    users=test.select('user_id').distinct(),
    items=test.select('item_id').distinct(),
    context='no_context',
    log=train,
    user_features=None,
    item_features=None,
    filter_seen_items=True
)

/home/17110781/projects/sponge-bob-magic/venv/lib/python3.7/site-packages/pandas/core/frame.py:7138: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,


CPU times: user 1.22 s, sys: 92 ms, total: 1.31 s
Wall time: 20.6 s


In [10]:
# в библиотеке реализовано несколько различных метрик качества рекомендательных систем, встречающихся в литературе
# пользователь может выбрать те, которые лучше подходят бизнес-задаче
from sponge_bob_magic.metrics import HitRate

HitRate()(recs, test, k=10)

0.015810276679841896

In [11]:
from sponge_bob_magic.metrics import NDCG

NDCG()(recs, test, k=10)

0.007158489468963139

## ALS

In [12]:
# библиотека также содержит классические алгоритмы рекомендаций, например, матричную факторизацию
from sponge_bob_magic.models.als_recommender import ALSRecommender

als = ALSRecommender(rank=100)

In [13]:
%%time
als.fit(
    log=train,
    user_features=None,
    item_features=None
)

CPU times: user 20 ms, sys: 12 ms, total: 32 ms
Wall time: 35.2 s


In [14]:
%%time
recs = als.predict(
    k=10,
    users=test.select('user_id').distinct(),
    items=test.select('item_id').distinct(),
    context='no_context',
    log=train,
    user_features=None,
    item_features=None,
    filter_seen_items=True
)

CPU times: user 20 ms, sys: 20 ms, total: 40 ms
Wall time: 26.2 s


In [15]:
HitRate()(recs, test, k=10)

0.07114624505928854

In [16]:
NDCG()(recs, test, k=10)

0.03175415554570095

In [17]:
## Своя модель

In [18]:
# для построения своей модели нужно использовать тот же самый split, что и для бейзлайнов
train.toPandas().to_csv("train.csv", index=False)

In [19]:
!head train.csv

user_id,item_id,relevance,timestamp,context
1090,2273,4.0,2000-11-23 01:33:35,no_context
1090,1645,4.0,2000-11-23 01:33:11,no_context
1090,100,3.0,2000-11-23 01:32:42,no_context
1090,1894,1.0,2000-11-23 01:31:32,no_context
1090,47,4.0,2000-11-23 01:31:17,no_context
1090,3176,3.0,2000-11-23 01:30:21,no_context
1090,2763,4.0,2000-11-23 01:30:21,no_context
1090,2762,5.0,2000-11-23 01:29:56,no_context
1090,2396,2.0,2000-11-23 01:29:39,no_context


In [20]:
# здесь магия: можно взять train и обучить на нём свою любимую модель вне библиотеки
# также нужно выдать рекомендации обученной моделью

In [21]:
recs.toPandas().to_csv("recs.csv", index=False)

In [22]:
# теперь нужно прочитать рекомендации в формате, поддерживаемом библиотекой
recs = DataPreparator(spark).transform_log(
    "recs.csv",
    columns_names={
        "user_id": "user_id",
        "item_id": "item_id",
        "relevance": "relevance"
    },
    header=True,
    format_type="csv"
)

In [23]:
# и сравнить качество своей модели с бейзлайнамии
HitRate()(recs, test, k=10)

0.07114624505928854